In [1]:
import pandas as pd 
import numpy as np
df = pd.read_csv('C:\\Users\\Dell\\Downloads\\Spotify_Youtube.csv') 
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20718 entries, 0 to 20717
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        20718 non-null  int64  
 1   Artist            20718 non-null  object 
 2   Url_spotify       20718 non-null  object 
 3   Track             20718 non-null  object 
 4   Album             20718 non-null  object 
 5   Album_type        20718 non-null  object 
 6   Uri               20718 non-null  object 
 7   Danceability      20716 non-null  float64
 8   Energy            20716 non-null  float64
 9   Key               20716 non-null  float64
 10  Loudness          20716 non-null  float64
 11  Speechiness       20716 non-null  float64
 12  Acousticness      20716 non-null  float64
 13  Instrumentalness  20716 non-null  float64
 14  Liveness          20716 non-null  float64
 15  Valence           20716 non-null  float64
 16  Tempo             20716 non-null  float6

In [2]:
df.columns

Index(['Unnamed: 0', 'Artist', 'Url_spotify', 'Track', 'Album', 'Album_type',
       'Uri', 'Danceability', 'Energy', 'Key', 'Loudness', 'Speechiness',
       'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo',
       'Duration_ms', 'Url_youtube', 'Title', 'Channel', 'Views', 'Likes',
       'Comments', 'Description', 'Licensed', 'official_video', 'Stream'],
      dtype='object')

In [3]:
df = df.drop(columns=['Loudness','Speechiness','Acousticness','Instrumentalness', 'Liveness', 'Valence', 'Tempo',
       'Duration_ms', 'Licensed', 'Uri',
       'Key'])

In [4]:
df.columns

Index(['Unnamed: 0', 'Artist', 'Url_spotify', 'Track', 'Album', 'Album_type',
       'Danceability', 'Energy', 'Url_youtube', 'Title', 'Channel', 'Views',
       'Likes', 'Comments', 'Description', 'official_video', 'Stream'],
      dtype='object')

In [5]:
df.isnull().sum()

Unnamed: 0          0
Artist              0
Url_spotify         0
Track               0
Album               0
Album_type          0
Danceability        2
Energy              2
Url_youtube       470
Title             470
Channel           470
Views             470
Likes             541
Comments          569
Description       876
official_video    470
Stream            576
dtype: int64

In [6]:
df = df.drop_duplicates(subset='Title', keep="first") 

In [7]:
df.columns

Index(['Unnamed: 0', 'Artist', 'Url_spotify', 'Track', 'Album', 'Album_type',
       'Danceability', 'Energy', 'Url_youtube', 'Title', 'Channel', 'Views',
       'Likes', 'Comments', 'Description', 'official_video', 'Stream'],
      dtype='object')

In [8]:
import warnings
warnings.filterwarnings("ignore")

# Importing modules
import pandas as pd
import numpy as np
import os
import re

# LDA Model
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
from pprint import pprint
from gensim.models import CoherenceModel
import spacy
from nltk.corpus import stopwords

# Import the wordcloud library
from wordcloud import WordCloud
!pip install pyLDAvis


# Visualize the topics
import pyLDAvis
import pyLDAvis.gensim_models
import pickle 
import pyLDAvis

In [9]:
df['overview'] = df['Channel']+" "+ df['Track']+" "+df['Description']

In [10]:
df.head()

,Unnamed: 0,Artist,Url_spotify,Track,Album,Album_type,Danceability,Energy,Url_youtube,Title,Channel,Views,Likes,Comments,Description,official_video,Stream,overview
0,0,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,Feel Good Inc.,Demon Days,album,0.818,0.705,https://www.youtube.com/watch?v=HyHNuVaZJ-k,Gorillaz - Feel Good Inc. (Official Video),Gorillaz,693555221.0,6220896.0,169907.0,Official HD Video for Gorillaz' fantastic trac...,True,1.040235e+09,Gorillaz Feel Good Inc. Official HD Video for ...
1,1,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,Rhinestone Eyes,Plastic Beach,album,0.676,0.703,https://www.youtube.com/watch?v=yYDmaexVHic,Gorillaz - Rhinestone Eyes [Storyboard Film] (...,Gorillaz,72011645.0,1079128.0,31003.0,The official video for Gorillaz - Rhinestone E...,True,3.100837e+08,Gorillaz Rhinestone Eyes The official video fo...
2,2,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,New Gold (feat. Tame Impala and Bootie Brown),New Gold (feat. Tame Impala and Bootie Brown),single,0.695,0.923,https://www.youtube.com/watch?v=qJa-VFwPpYA,Gorillaz - New Gold ft. Tame Impala & Bootie B...,Gorillaz,8435055.0,282142.0,7399.0,Gorillaz - New Gold ft. Tame Impala & Bootie B...,True,6.306347e+07,Gorillaz New Gold (feat. Tame Impala and Booti...
3,3,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,On Melancholy Hill,Plastic Beach,album,0.689,0.739,https://www.youtube.com/watch?v=04mfKJWDSzI,Gorillaz - On Melancholy Hill (Official Video),Gorillaz,211754952.0,1788577.0,55229.0,Follow Gorillaz online:\nhttp://gorillaz.com \...,True,4.346636e+08,Gorillaz On Melancholy Hill Follow Gorillaz on...
4,4,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,Clint Eastwood,Gorillaz,album,0.663,0.694,https://www.youtube.com/watch?v=1V_xRb0x9aw,Gorillaz - Clint Eastwood (Official Video),Gorillaz,618480958.0,6197318.0,155930.0,The official music video for Gorillaz - Clint ...,True,6.172597e+08,Gorillaz Clint Eastwood The official music vid...


In [11]:
# Tokenization
def sentence_to_words(sentences):
 for sentence in sentences:
  yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) # deacc=True removes punctuations and special characters

all_words = list(sentence_to_words(df['overview']))


# remove stopwords
import nltk
nltk.download('stopwords')

stop_words = stopwords.words('english')

# stop_words.extend(['']) #extend existing stop word list if needed
def remove_stopwords(texts):
 return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]


# Building  the bigram 
bigram = gensim.models.Phrases(all_words, min_count=5, threshold=10) 

bigram_mod = gensim.models.phrases.Phraser(bigram)
def create_bigrams(texts):
 return [bigram_mod[doc] for doc in texts]

# Lemmatization
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
 texts_out = []
 for sent in texts:
  doc = nlp(" ".join(sent)) 
  texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
 return texts_out

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
all_words_nostops = remove_stopwords(all_words)
all_words_bigrams = create_bigrams(all_words_nostops)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
data_lemmatized = lemmatization(all_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1]) 

[['official_hd', 'video', 'gorillaz', 'fantastic', 'track', 'follow', 'gorillaz', 'com', 'http', 'gorillaz', 'http', 'gorillazband', 'information', 'gorillaz', 'check', 'official_website', 'http', 'www', 'com']]


In [15]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Filter out tokens that appear in only 1 documents and appear in more than 90% of the documents
id2word.filter_extremes(no_below=2, no_above=0.9)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [17]:
# Building LDA model for 10 topics
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=20, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [25]:
# Printing the Keywords in the 20 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.039*"found_ee" + 0.018*"fire" + 0.014*"rain" + 0.013*"last" + '
  '0.012*"wish" + 0.011*"beautiful" + 0.011*"https" + 0.011*"sleep" + '
  '0.010*"sky" + 0.009*"day"'),
 (1,
  '0.164*"https" + 0.132*"com" + 0.053*"http" + 0.048*"twitter" + '
  '0.043*"youtube" + 0.042*"instagram" + 0.037*"spotify" + 0.033*"facebook" + '
  '0.030*"follow" + 0.024*"music"'),
 (2,
  '0.065*"director" + 0.042*"production" + 0.038*"producer" + 0.027*"produce" '
  '+ 0.027*"editor" + 0.024*"cast" + 0.023*"studio" + '
  '0.022*"executive_producer" + 0.019*"film" + 0.017*"photography"'),
 (3,
  '0.070*"get" + 0.018*"girl" + 0.015*"come" + 0.011*"go" + 0.010*"bitch" + '
  '0.010*"fuck" + 0.010*"baby" + 0.009*"shit" + 0.009*"make" + 0.008*"put"'),
 (4,
  '0.110*"http_bit" + 0.050*"ly" + 0.048*"music" + 0.043*"song" + '
  '0.017*"movie" + 0.016*"lyric" + 0.016*"set" + 0.014*"video" + '
  '0.013*"singer" + 0.013*"twitter"'),
 (5,
  '0.056*"https" + 0.037*"video" + 0.026*"perform" + 0.026*"social" + '
  '0

In [26]:
def recommend_by_artist(artist, df):

    recommended = []
    top10_list = []
    
    artist = artist.lower()
    df['artist'] = df['artist'].str.lower()
    
    topic_number = df[df['artist']==artist].Topic.values
    doc_number = df[df['artist']==artist].Doc.values    
        
    output = df[df['Topic']==topic_number[0]].sort_values('Probability', ascending=False).reset_index(drop=True)

    index = output[output['Doc']==doc_number[0]].index[0]
    
    top10_list += list(output.iloc[index-5:index].index)
    top10_list += list(output.iloc[index+1:index+6].index)
    
    output['artist'] = output['atist'].str.artist()
    
    for each in top10_list:
        recommended.append(output.iloc[each].artist)
        
    return recommended

In [27]:
recommend_by_artist('thasup', df)

KeyError: 'artist'